Data uploading

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


Library loading

In [ ]:
!pip install deap

     |████████████████████████████████| 163kB 30.1MB/s 


In [ ]:
import pandas as pd
import numpy as np
import random as rnd
from sklearn.metrics import classification_report
import io

from deap import base
from deap import creator
from deap import tools

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.pipeline import make_pipeline

In [ ]:
df = pd.read_csv(io.BytesIO(uploaded['biodeg.csv']), sep=';')
df_name = df.columns
df.info()

KeyError: ignored

In [ ]:
SEED = 42
np.random.seed(SEED)

### Genetic Algorithm

In [ ]:
columns = df.drop('experimental_class', axis=1).columns
training = pd.concat([df])

1. Logistic Regression

In [ ]:
def getPenaltyLR():
  penalties = ['l1', 'l2']
  index = rnd.randint(0, len(penalties) - 1)   
  penalty = penalties[index]

  return penalty

def getCLR():
  r = rnd.random()
  r2 = rnd.randint(0, 4)
  C = r + r2 + 0.000000001 

  return C

def getMultiClassLR():
  multi_classes = ['auto', 'ovr', 'multinomial']
  index = rnd.randint(0, len(multi_classes) - 1)   
  multi_class = multi_classes[index]

  return multi_class

In [ ]:
def getModelLR(individual):
  penalty = individual[0]
  C = individual[1]
  multi_class = individual[2]

  model = LogisticRegression(penalty=penalty, C=C, multi_class=multi_class)

  return model

def getXyLR(individual):
  selected_columns = list(columns)

  for i in range(len(individual[3:])):
    if individual[3 + i] < 1:
      selected_columns.remove(columns[i])

  target_columns = np.append(['experimental_class'], selected_columns)
  df = training.loc[:, target_columns].dropna()
  X  = df.loc[:, selected_columns]
  y = np.ravel(df.loc[:, ['experimental_class']])
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
  standard_scaler = preprocessing.StandardScaler().fit(X)
  X_train = standard_scaler.transform(X_train)
  X_test = standard_scaler.transform(X_test)
  return [X_train, y_train, X_test, y_test, selected_columns, standard_scaler] 

def evaluteOneMaxLR(individual):
  model = getModelLR(individual)
  Xy = getXyLR(individual)
  scores = cross_val_score(model, Xy[0], Xy[1], cv=5, n_jobs=-1)
  mean = scores.mean(),
  
  return mean

def myMutateLR(individual, indpb=0.05):
  if rnd.random() < indpb:
      individual[0] = toolbox.attribute_PenaltyLR()

  if rnd.random() < indpb:
      individual[1] = toolbox.attribute_CLR()

  if rnd.random() < indpb:
      individual[2] = toolbox.attribute_Multi_ClassLR()
      
  for i in range(len(individual[3:])):
      if rnd.random() < indpb:
          individual[3 + i] = toolbox.attribute_bool()

In [ ]:
creator.create('FitnessMax', base.Fitness, weights=(1.0,))
creator.create('Individual', list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register('attribute_bool', rnd.randint, 0, 1)
toolbox.register('attribute_PenaltyLR', getPenaltyLR)
toolbox.register('attribute_CLR', getCLR)
toolbox.register('attribute_Multi_ClassLR', getMultiClassLR)

function_sequence = [toolbox.attribute_PenaltyLR, toolbox.attribute_CLR, toolbox.attribute_Multi_ClassLR]

for column in columns:
  function_sequence.append(toolbox.attribute_bool)

toolbox.register('individual', tools.initCycle, creator.Individual, function_sequence, 1)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)

In [ ]:
CXPB, MUTPB, NGEN, POPSIZE = 0.5, 0.2, 100, 100
pop = toolbox.population(n=POPSIZE) 

In [ ]:
toolbox.register('evaluate', evaluteOneMaxLR)
toolbox.register('mate', tools.cxTwoPoint)
toolbox.register('mutate', myMutateLR, indpb=0.15)
toolbox.register('select', tools.selTournament, tournsize=3)
rnd.seed(66)

In [ ]:
print('Start evolution of Logistic Regression')

fitnesses = list(map(toolbox.evaluate, pop))
for individual, fitness in zip(pop, fitnesses):
  individual.fitness.values = fitness

print (f'  Evaluated {len(pop)} individuals')

for generation in range(NGEN):
  print(f'-- Generation {generation} --')

  offspring = toolbox.select(pop, len(pop))
  offspring = list(map(toolbox.clone, offspring))

  for child1, child2 in zip(offspring[::2], offspring[1::2]):
    if rnd.random() < CXPB:
      c1 = toolbox.clone(child1)
      c2 = toolbox.clone(child2)
      toolbox.mate(child1, child2)
      if c1 != child1: 
        del child1.fitness.values
      if c2 != child2: 
        del child2.fitness.values

  for mutant in offspring:
    if rnd.random() < MUTPB:
      m1 = toolbox.clone(mutant)
      toolbox.mutate(mutant)
      if m1 != mutant: 
        del mutant.fitness.values

  invalid_individual = [individual for individual in offspring if not individual.fitness.valid]
  fitnesses = map(toolbox.evaluate, invalid_individual)

  for individual, fitness in zip(invalid_individual, fitnesses):
    individual.fitness.values = fitness

  print (f'  Evaluated {len(invalid_individual)} individuals')

  pop[:] = offspring
  fits = [individual.fitness.values[0] for individual in pop]

  length = len(pop)
  mean = sum(fits) / length
  sum2 = sum(x * x for x in fits)
  std = abs(sum2 / length - mean**2)**0.5
  best_individual = tools.selBest(pop, POPSIZE)[0]
  print(f'Best individual is {best_individual}, {best_individual.fitness.values}')        
  print(f'  Min: {min(fits)}')
  print(f'  Max: {max(fits)}')
  print(f'  Avg: {mean}')
  print(f'  Std: {std}')

print('End evolution of Logistic Regression')

best_individual = tools.selBest(pop, POPSIZE)[0]
print(f'Best individual is {best_individual}, {best_individual.fitness.values}')

In [ ]:
model = getModelLR(best_individual)
Xy = getXyLR(best_individual)
LR_columns = Xy[4]
scaler = Xy[5]
print(f'Selected Features: {LR_columns}')

In [ ]:
X_train = Xy[0]
Y_train = Xy[1]
X_test = Xy[2]
Y_test = Xy[3]

scores = cross_val_score(model, X_train, Y_train, cv=5).mean()
cv_LRGA_score = scores.mean()
print(f'LRGA CV score: {scores.mean()}')
model.fit(X_train ,Y_train)
print(f'Training score: {model.score(X_train, Y_train)}')
prediction = model.predict(X_test)
print(f'Prediction score: {(prediction == Y_test).mean()}')
print('\n' + classification_report(Y_test, prediction))

2. K-Nearest Neighbor (KNN)

In [ ]:
def getNeighborKNN():
  neighbor = rnd.randint(1, 20)

  return neighbor

def getWeightKNN():
  weights = ['uniform', 'distance']
  index = rnd.randint(0, len(weights) - 1)   
  weight = weights[index]

  return weight

In [ ]:
def getModelKNN(individual):
  n_neighbors = individual[0]
  weights = individual[1]

  model = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights)

  return model

def getXyKNN(individual):
  selected_columns = list(columns)

  for i in range(len(individual[2:])):
    if individual[2 + i] < 1:
      selected_columns.remove(columns[i])

  target_columns = np.append(['experimental_class'], selected_columns)
  df = training.loc[:, target_columns].dropna()
  X  = df.loc[:, selected_columns]
  y = np.ravel(df.loc[:, ['experimental_class']])
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
  standard_scaler = preprocessing.StandardScaler().fit(X)
  X_train = standard_scaler.transform(X_train)
  X_test = standard_scaler.transform(X_test)
  return [X_train, y_train, X_test, y_test, selected_columns, standard_scaler] 

def evaluteOneMaxKNN(individual):
  model = getModelKNN(individual)
  Xy = getXyKNN(individual)
  scores = cross_val_score(model, Xy[0], Xy[1], cv=5, n_jobs=-1)
  mean = scores.mean(),
  
  return mean

def myMutateKNN(individual, indpb=0.05):
  if rnd.random() < indpb:
      individual[0] = toolbox.attribute_N_NeighborsKNN()

  if rnd.random() < indpb:
      individual[1] = toolbox.attribute_WeightsKNN()
      
  for i in range(len(individual[2:])):
      if rnd.random() < indpb:
          individual[2 + i] = toolbox.attribute_bool()

In [ ]:
creator.create('FitnessMax', base.Fitness, weights=(1.0,))
creator.create('Individual', list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register('attribute_bool', rnd.randint, 0, 1)
toolbox.register('attribute_N_NeighborsKNN', getNeighborKNN)
toolbox.register('attribute_WeightsKNN', getWeightKNN)

function_sequence = [toolbox.attribute_N_NeighborsKNN, toolbox.attribute_WeightsKNN]

for column in columns:
  function_sequence.append(toolbox.attribute_bool)

toolbox.register('individual', tools.initCycle, creator.Individual, function_sequence, 1)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)

In [ ]:
toolbox.register('evaluate', evaluteOneMaxKNN)
toolbox.register('mate', tools.cxTwoPoint)
toolbox.register('mutate', myMutateKNN, indpb=0.15)
toolbox.register('select', tools.selTournament, tournsize=3)
rnd.seed(66)

In [ ]:
CXPB, MUTPB, NGEN, POPSIZE = 0.5, 0.2, 100, 100
pop = toolbox.population(n=POPSIZE) 

In [ ]:
print('Start evolution of K-Nearest Neighbor')

fitnesses = list(map(toolbox.evaluate, pop))
for individual, fitness in zip(pop, fitnesses):
  individual.fitness.values = fitness

print (f'  Evaluated {len(pop)} individuals')

for generation in range(NGEN):
  print(f'-- Generation {generation} --')

  offspring = toolbox.select(pop, len(pop))
  offspring = list(map(toolbox.clone, offspring))

  for child1, child2 in zip(offspring[::2], offspring[1::2]):
    if rnd.random() < CXPB:
      c1 = toolbox.clone(child1)
      c2 = toolbox.clone(child2)
      toolbox.mate(child1, child2)
      if c1 != child1: 
        del child1.fitness.values
      if c2 != child2: 
        del child2.fitness.values

  for mutant in offspring:
    if rnd.random() < MUTPB:
      m1 = toolbox.clone(mutant)
      toolbox.mutate(mutant)
      if m1 != mutant: 
        del mutant.fitness.values

  invalid_individual = [individual for individual in offspring if not individual.fitness.valid]
  fitnesses = map(toolbox.evaluate, invalid_individual)

  for individual, fitness in zip(invalid_individual, fitnesses):
    individual.fitness.values = fitness

  print (f'  Evaluated {len(invalid_individual)} individuals')

  pop[:] = offspring
  fits = [individual.fitness.values[0] for individual in pop]

  length = len(pop)
  mean = sum(fits) / length
  sum2 = sum(x * x for x in fits)
  std = abs(sum2 / length - mean**2)**0.5
  best_individual = tools.selBest(pop, POPSIZE)[0]
  print(f'Best individual is {best_individual}, {best_individual.fitness.values}')        
  print(f'  Min: {min(fits)}')
  print(f'  Max: {max(fits)}')
  print(f'  Avg: {mean}')
  print(f'  Std: {std}')

print('End evolution of K-Nearest Neighbor')

best_individual = tools.selBest(pop, POPSIZE)[0]
print(f'Best individual is {best_individual}, {best_individual.fitness.values}')

In [ ]:
model = getModelKNN(best_individual)
Xy = getXyKNN(best_individual)
KNN_columns = Xy[4]
scaler = Xy[5]
print(f'Selected Features: {KNN_columns}')

In [ ]:
X_train = Xy[0]
Y_train = Xy[1]
X_test = Xy[2]
Y_test = Xy[3]

scores = cross_val_score(model, X_train, Y_train, cv=5).mean()
cv_KNNGA_score = scores.mean()
print(f'KNNGA CV score: {scores.mean()}')
model.fit(X_train ,Y_train)
print(f'Training score: {model.score(X_train, Y_train)}')
prediction = model.predict(X_test)
print(f'Prediction score: {(prediction == Y_test).mean()}')
print('\n' + classification_report(Y_test, prediction))

3. Support Vector Machine (SVM)

In [ ]:
def getCSVM():
  r = rnd.random()
  r2 = rnd.randint(0, 2)
  C = r + r2 + 0.000000001 

  return C

def getGammaSVM():
  r = rnd.random()
  r2 = rnd.randint(0, 3)
  gamma = 0.000000001 + (r / (10**r2))

  return gamma

def getKernelSVM():
  kernels = ['rbf', 'linear', 'sigmoid']
  index = rnd.randint(0, len(kernels) - 1)   
  kernel = kernels[index]

  return kernel

In [ ]:
def getModelSVM(individual):
  C = individual[0]
  gamma = individual[1]
  kernel = individual[2]
  
  model = SVC(kernel=kernel, C=C, gamma=gamma)

  return model

def getXySVM(individual):
  selected_columns = list(columns)

  for i in range(len(individual[3:])):
    if individual[3 + i] < 1:
      selected_columns.remove(columns[i])

  target_columns = np.append(['experimental_class'], selected_columns)
  df = training.loc[:, target_columns].dropna()
  X  = df.loc[:, selected_columns]
  y = np.ravel(df.loc[:, ['experimental_class']])
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
  standard_scaler = preprocessing.StandardScaler().fit(X)
  X_train = standard_scaler.transform(X_train)
  X_test = standard_scaler.transform(X_test)
  return [X_train, y_train, X_test, y_test, selected_columns, standard_scaler] 

def evaluteOneMaxSVM(individual):
  model = getModelSVM(individual)
  Xy = getXySVM(individual)
  scores = cross_val_score(model, Xy[0], Xy[1], cv=5, n_jobs=-1)
  mean = scores.mean(),
  
  return mean

def myMutateSVM(individual, indpb=0.05):
  if rnd.random() < indpb:
      individual[0] = toolbox.attribute_CKNN()

  if rnd.random() < indpb:
      individual[1] = toolbox.attribute_GammaKNN()

  if rnd.random() < indpb:
      individual[2] = toolbox.attribute_KernelSVM()
      
  for i in range(len(individual[3:])):
      if rnd.random() < indpb:
          individual[3 + i] = toolbox.attribute_bool()

In [ ]:
creator.create('FitnessMax', base.Fitness, weights=(1.0,))
creator.create('Individual', list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register('attribute_bool', rnd.randint, 0, 1)
toolbox.register('attribute_CKNN', getCSVM)
toolbox.register('attribute_GammaKNN', getGammaSVM)
toolbox.register('attribute_KernelSVM', getKernelSVM)

function_sequence = [toolbox.attribute_CKNN, toolbox.attribute_GammaKNN, toolbox.attribute_KernelSVM]

for column in columns:
  function_sequence.append(toolbox.attribute_bool)

toolbox.register('individual', tools.initCycle, creator.Individual, function_sequence, 1)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)

In [ ]:
toolbox.register('evaluate', evaluteOneMaxSVM)
toolbox.register('mate', tools.cxTwoPoint)
toolbox.register('mutate', myMutateSVM, indpb=0.15)
toolbox.register('select', tools.selTournament, tournsize=3)
rnd.seed(66)

In [ ]:
CXPB, MUTPB, NGEN, POPSIZE = 0.5, 0.2, 100, 100
pop = toolbox.population(n=POPSIZE) 

In [ ]:
print('Start evolution of Support Vector Machine')

fitnesses = list(map(toolbox.evaluate, pop))
for individual, fitness in zip(pop, fitnesses):
  individual.fitness.values = fitness

print (f'  Evaluated {len(pop)} individuals')

for generation in range(NGEN):
  print(f'-- Generation {generation} --')

  offspring = toolbox.select(pop, len(pop))
  offspring = list(map(toolbox.clone, offspring))

  for child1, child2 in zip(offspring[::2], offspring[1::2]):
    if rnd.random() < CXPB:
      c1 = toolbox.clone(child1)
      c2 = toolbox.clone(child2)
      toolbox.mate(child1, child2)
      if c1 != child1: 
        del child1.fitness.values
      if c2 != child2: 
        del child2.fitness.values

  for mutant in offspring:
    if rnd.random() < MUTPB:
      m1 = toolbox.clone(mutant)
      toolbox.mutate(mutant)
      if m1 != mutant: 
        del mutant.fitness.values

  invalid_individual = [individual for individual in offspring if not individual.fitness.valid]
  fitnesses = map(toolbox.evaluate, invalid_individual)

  for individual, fitness in zip(invalid_individual, fitnesses):
    individual.fitness.values = fitness

  print (f'  Evaluated {len(invalid_individual)} individuals')

  pop[:] = offspring
  fits = [individual.fitness.values[0] for individual in pop]

  length = len(pop)
  mean = sum(fits) / length
  sum2 = sum(x * x for x in fits)
  std = abs(sum2 / length - mean**2)**0.5
  best_individual = tools.selBest(pop, POPSIZE)[0]
  print(f'Best individual is {best_individual}, {best_individual.fitness.values}')        
  print(f'  Min: {min(fits)}')
  print(f'  Max: {max(fits)}')
  print(f'  Avg: {mean}')
  print(f'  Std: {std}')

print('End evolution of Support Vector Machine')

best_individual = tools.selBest(pop, POPSIZE)[0]
print(f'Best individual is {best_individual}, {best_individual.fitness.values}')

In [ ]:
model = getModelSVM(best_individual)
Xy = getXySVM(best_individual)
SVM_columns = Xy[4]
scaler = Xy[5]
print(f'Selected Features: {SVM_columns}')

In [ ]:
X_train = Xy[0]
Y_train = Xy[1]
X_test = Xy[2]
Y_test = Xy[3]

scores = cross_val_score(model, X_train, Y_train, cv=5).mean()
cv_SVMGA_score = scores.mean()
print(f'SVMGA CV score: {scores.mean()}')
model.fit(X_train ,Y_train)
print(f'Training score: {model.score(X_train, Y_train)}')
prediction = model.predict(X_test)
print(f'Prediction score: {(prediction == Y_test).mean()}')
print('\n' + classification_report(Y_test, prediction))

4. Decision Tree

In [ ]:
def getMaxDepthDT():
  max_depth = rnd.randint(3, 4)
  return max_depth

def getMaxFeaturesDT():
  max_features = rnd.randint(1, 4)
  return max_features

def getMinSamplesLeafDT():
  min_samples_leaf = rnd.randint(1, 4)
  return min_samples_leaf

def getMinSamplesSplitDT():
  min_samples_split = rnd.randint(1, 20)
  return min_samples_split

def getCriterionDT():
  criteria = ['gini', 'entropy']
  index = rnd.randint(0, len(criteria) - 1)   
  criterion = criteria[index]

  return criterion

In [ ]:
def getModelDT(individual):
  max_depth = individual[0]
  max_features = individual[1]
  min_samples_leaf = individual[2]
  min_samples_split = individual[3]
  criterion = individual[4]
  
  model = DecisionTreeClassifier(max_depth=max_depth, max_features=max_features, min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split, criterion=criterion)

  return model

def getXyDT(individual):
  selected_columns = list(columns)

  for i in range(len(individual[5:])):
    if individual[5 + i] < 1:
      selected_columns.remove(columns[i])

  target_columns = np.append(['experimental_class'], selected_columns)
  df = training.loc[:, target_columns].dropna()
  X  = df.loc[:, selected_columns]
  y = np.ravel(df.loc[:, ['experimental_class']])
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
  standard_scaler = preprocessing.StandardScaler().fit(X)
  X_train = standard_scaler.transform(X_train)
  X_test = standard_scaler.transform(X_test)
  return [X_train, y_train, X_test, y_test, selected_columns, standard_scaler]  

def evaluteOneMaxDT(individual):
  model = getModelDT(individual)
  Xy = getXyDT(individual)
  scores = cross_val_score(model, Xy[0], Xy[1], cv=5, n_jobs=-1)
  mean = scores.mean(),
  
  return mean

def myMutateDT(individual, indpb=0.05):
  if rnd.random() < indpb:
      individual[0] = toolbox.attribute_Max_Depth_DT()

  if rnd.random() < indpb:
      individual[1] = toolbox.attribute_Max_FeaturesDT()

  if rnd.random() < indpb:
      individual[2] = toolbox.attribute_Min_Samples_LeafDT()
  
  if rnd.random() < indpb:
      individual[2] = toolbox.attribute_Min_Samples_SplitDT()

  if rnd.random() < indpb:
    individual[2] = toolbox.attribute_Min_Samples_SplitDT()
      
  for i in range(len(individual[5:])):
      if rnd.random() < indpb:
          individual[5 + i] = toolbox.attribute_bool()

In [ ]:
creator.create('FitnessMax', base.Fitness, weights=(1.0,))
creator.create('Individual', list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register('attribute_bool', rnd.randint, 0, 1)
toolbox.register('attribute_Max_Depth_DT', getMaxDepthDT)
toolbox.register('attribute_Max_FeaturesDT', getMaxFeaturesDT)
toolbox.register('attribute_Min_Samples_LeafDT', getMinSamplesLeafDT)
toolbox.register('attribute_Min_Samples_SplitDT', getMinSamplesSplitDT)
toolbox.register('attribute_CriterionDT', getCriterionDT)

function_sequence = [toolbox.attribute_Max_Depth_DT, toolbox.attribute_Max_FeaturesDT, toolbox.attribute_Min_Samples_LeafDT, toolbox.attribute_Min_Samples_SplitDT, toolbox.attribute_CriterionDT]

for column in columns:
  function_sequence.append(toolbox.attribute_bool)

toolbox.register('individual', tools.initCycle, creator.Individual, function_sequence, 1)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)

In [ ]:
toolbox.register('evaluate', evaluteOneMaxDT)
toolbox.register('mate', tools.cxTwoPoint)
toolbox.register('mutate', myMutateDT, indpb=0.15)
toolbox.register('select', tools.selTournament, tournsize=3)
rnd.seed(66)

In [ ]:
CXPB, MUTPB, NGEN, POPSIZE = 0.5, 0.2, 100, 100
pop = toolbox.population(n=POPSIZE) 

In [ ]:
print('Start evolution of Decision Tree')

fitnesses = list(map(toolbox.evaluate, pop))
for individual, fitness in zip(pop, fitnesses):
  individual.fitness.values = fitness

print (f'  Evaluated {len(pop)} individuals')

for generation in range(NGEN):
  print(f'-- Generation {generation} --')

  offspring = toolbox.select(pop, len(pop))
  offspring = list(map(toolbox.clone, offspring))

  for child1, child2 in zip(offspring[::2], offspring[1::2]):
    if rnd.random() < CXPB:
      c1 = toolbox.clone(child1)
      c2 = toolbox.clone(child2)
      toolbox.mate(child1, child2)
      if c1 != child1: 
        del child1.fitness.values
      if c2 != child2: 
        del child2.fitness.values

  for mutant in offspring:
    if rnd.random() < MUTPB:
      m1 = toolbox.clone(mutant)
      toolbox.mutate(mutant)
      if m1 != mutant: 
        del mutant.fitness.values

  invalid_individual = [individual for individual in offspring if not individual.fitness.valid]
  fitnesses = map(toolbox.evaluate, invalid_individual)

  for individual, fitness in zip(invalid_individual, fitnesses):
    individual.fitness.values = fitness

  print (f'  Evaluated {len(invalid_individual)} individuals')

  pop[:] = offspring
  fits = [individual.fitness.values[0] for individual in pop]

  length = len(pop)
  mean = sum(fits) / length
  sum2 = sum(x * x for x in fits)
  std = abs(sum2 / length - mean**2)**0.5
  best_individual = tools.selBest(pop, POPSIZE)[0]
  print(f'Best individual is {best_individual}, {best_individual.fitness.values}')        
  print(f'  Min: {min(fits)}')
  print(f'  Max: {max(fits)}')
  print(f'  Avg: {mean}')
  print(f'  Std: {std}')

print('End evolution of Decision Tree')

best_individual = tools.selBest(pop, POPSIZE)[0]
print(f'Best individual is {best_individual}, {best_individual.fitness.values}')

In [ ]:
model = getModelDT(best_individual)
Xy = getXyDT(best_individual)
DT_columns = Xy[4]
scaler = Xy[5]
print(f'Selected Features: {DT_columns}')

In [ ]:
X_train = Xy[0]
Y_train = Xy[1]
X_test = Xy[2]
Y_test = Xy[3]

scores = cross_val_score(model, X_train, Y_train, cv=5).mean()
cv_DTGA_score = scores.mean()
print(f'DTGA CV score: {scores.mean()}')
model.fit(X_train ,Y_train)
print(f'Training score: {model.score(X_train, Y_train)}')
prediction = model.predict(X_test)
print(f'Prediction score: {(prediction == Y_test).mean()}')
print('\n' + classification_report(Y_test, prediction))